In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
data_import = pd.read_csv('/Users/sitanshurupani/Data Science Coursework/Sem 4/GQP - National Parks/DataSpell Project/Data/Daily_attendance.csv')

In [4]:
all_years_test = data_import
all_years_test.set_index('Full_Date', inplace=True)

In [5]:
all_years_test.index = pd.to_datetime(all_years_test.index)
ds = all_years_test.index.to_series()
all_years_test['YEAR'] = ds.dt.year
all_years_test['MONTH'] = ds.dt.month
all_years_test['DAY_OF_WEEK'] = ds.dt.dayofweek
all_years_test['DAY'] = ds.dt.day

In [6]:
all_years_test.drop(columns=['snow_1h', 'temp_max', 'temp_min','YEAR'], inplace=True)
all_years_test["federal_holiday_flag"] = all_years_test["federal_holiday_flag"].astype(int)
all_years_test["school_holiday_flag"] = all_years_test["school_holiday_flag"].astype(int)

In [7]:
all_years_test['wind_gust'].fillna((all_years_test['wind_gust'].mean()), inplace=True)

In [8]:
all_years_test.isna().sum()

value                   0
temp                    0
pressure                0
humidity                0
wind_speed              0
wind_gust               0
rain_1h                 0
federal_holiday_flag    0
school_holiday_flag     0
MONTH                   0
DAY_OF_WEEK             0
DAY                     0
dtype: int64

In [9]:
mmsca = MinMaxScaler()
X_mmsca = pd.DataFrame(mmsca.fit_transform(all_years_test), columns=all_years_test.columns)
X_mmsca.index = all_years_test.index
X_mmsca.drop(columns=['value'], inplace=True)

In [10]:
X = X_mmsca.values
Y = all_years_test['value'].values

In [ ]:
preds, scores = [], []
kfold = KFold(n_splits=10, shuffle=True, random_state=1)
for train_idx, test_idx in kfold.split(X_mmsca):
    X_train, X_test = X[train_idx], X[test_idx]
    y_test = Y[test_idx]
    model = smf.glm(formula = "value ~ DAY  + DAY_OF_WEEK + MONTH + temp + pressure + humidity + wind_speed + wind_gust + rain_1h + federal_holiday_flag + school_holiday_flag",
                    data=X_train,
                    family=sm.families.NegativeBinomial()).fit()
    preds.append(model.get_prediction(X_test))
    scores.append(model.score(X_test, y_test))
print(scores)